In [ ]:
import numpy as np
import hyperspy.api as hys
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
import tifffile
plt.rcParams['font.family'] = 'Times New Roman'

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
si_adr = tkf.askopenfilename()
print(si_adr)

In [ ]:
si = hys.load(si_adr, signal_type="EELS")
print(si)

In [ ]:
ini_channel = 280.0
fin_channel = 315.0
dispersion = np.around(si.axes_manager[2].scale, 4)
print(ini_channel, fin_channel, dispersion)
e_range = np.arange(ini_channel, fin_channel, dispersion)
print(e_range.shape)

In [ ]:
si = si.isig[ini_channel:fin_channel]
data = si.data.clip(min=0.0)
si_shape = data.shape
print(si_shape)

data_flat = data.reshape(-1, si_shape[2])

In [ ]:
pi_onset = 282.0
pi_pos = 285.0
pi_final = 287.0
sigma_pos = 293.0
sigma_final = 315.0

In [ ]:
pi_ind = int((pi_pos - e_range[0]) / dispersion)
pi_onset_ind = int((pi_onset - e_range[0]) / dispersion)
pi_final_ind = int((pi_final - e_range[0]) / dispersion)
sigma_ind = int((sigma_pos - e_range[0]) / dispersion)
sigma_final_ind = int((sigma_final - e_range[0]) / dispersion)
print(pi_ind, pi_onset_ind, pi_final_ind, sigma_ind, sigma_final_ind)

pi_range = e_range[pi_onset_ind:pi_final_ind]
print(pi_range.shape)
print(pi_range[0], pi_range[-1])

pi_sigma_range = e_range[pi_onset_ind:sigma_final_ind]
print(pi_sigma_range.shape)
print(pi_sigma_range[0], pi_sigma_range[-1])

In [ ]:
peak_ratio_map = []
two_window_map = []
for i, s in enumerate(data_flat):
    tmp = s - np.min(s)
    pi_int = tmp[pi_ind]
    sigma_int = tmp[sigma_ind]
    if sigma_int != 0.0:
        peak_ratio_map.append(pi_int/sigma_int)
    else:
        peak_ratio_map.append(0)
    
    pi_spec = tmp[pi_onset_ind:pi_final_ind]
    pi_sigma_spec = tmp[pi_onset_ind:sigma_final_ind]
    
    Epi = np.trapz(pi_spec, pi_range)
    Episigma = np.trapz(pi_sigma_spec, pi_sigma_range)
    two_window_map.append(Epi/Episigma)

peak_ratio_map = np.asarray(peak_ratio_map).reshape(si_shape[0], si_shape[1])
print(peak_ratio_map.shape)
print(np.min(peak_ratio_map), np.max(peak_ratio_map))
print(np.mean(peak_ratio_map), np.median(peak_ratio_map))
two_window_map = np.asarray(two_window_map).reshape(si_shape[0], si_shape[1])
print(two_window_map.shape)
print(np.min(two_window_map), np.max(two_window_map))
print(np.mean(two_window_map), np.median(two_window_map))

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(peak_ratio_map, cmap="inferno", vmin=0.0, vmax=1.0)
ax[0].axis("off")
ax[1].imshow(two_window_map, cmap="inferno", vmin=0.03, vmax=0.10)
ax[1].axis("off")
fig.tight_layout()
plt.show()